<span STYLE="font-size:150%"> 
    Segment microCT scans
</span>

Docker image: gnasello/slicer-env:2023-07-06 \
Latest update: 10 March 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

In [1]:
# Monitor Memory in Slicer
import psutil, os
print(psutil.Process(os.getpid()).memory_info().rss / (1024**3), "GB used")

0.7232780456542969 GB used


# Load libraries

In [2]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [3]:
# this cell is tagged 'parameters'
volume_file = 'microCT_volume/microCT_volume.nrrd'
output_dir_path = 'segmented_volumes'
segmentMask_file = 'segmented_volumes/Segment_mask.seg.nrrd'
segments_greyvalues_file = 'segmented_volumes/segments_greyvalues.csv'

In [4]:
# Parameters
volume_file = "/config/Downloads/Gomez 5p1/microCT_volume/microCT_volume.nrrd"
output_dir_path = "/config/Downloads/Gomez 5p1/segmented_volumes/"
segmentMask_file = "/config/Downloads/Gomez 5p1/segmented_volumes/Segment_mask.seg.nrrd"


In [5]:
path = Path(volume_file)

# Remove image numbering _0000, _0001 ...
filename_output = path.stem[:-4]

In [6]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [7]:
## mm
masterVolumeNode.GetSpacing()

(0.009743730000000003, 0.009743730000000003, 0.009743730000000003)

Make ```segmented_volumes``` folder

In [8]:
output_directory = Path(output_dir_path)

output_directory.mkdir(parents=True, exist_ok=True)

In [9]:
# Monitor Memory in Slicer
import psutil, os
print(psutil.Process(os.getpid()).memory_info().rss / (1024**3), "GB used")

15.424633026123047 GB used


In [10]:
# Monitor Memory in Slicer
import psutil, os
print(psutil.Process(os.getpid()).memory_info().rss / (1024**3), "GB used")

15.430126190185547 GB used


# Create segmentationNode

## Load Segment Mask LabelmapNode

In [11]:
segmentationNode = slicer.util.loadSegmentation(segmentMask_file, properties={"name": "Segmentation"})

Rename segment of the segmentation node 

In [13]:
segmentation = segmentationNode.GetSegmentation()
segment = segmentation.GetNthSegment(0)
segment.SetName("SegmentMask")

Create temporary segment editor to get access to effects

In [14]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

## Set a mask in segmentation node
[link](https://discourse.slicer.org/t/masking-settings-from-a-script-on-a-segment-editor-effect-in-python/14323/7)

In [15]:
MaskSegment_name = 'SegmentMask'

MaskSegmentID = segmentation.GetSegmentIdBySegmentName(MaskSegment_name)
segmentEditorNode.SetMaskSegmentID(MaskSegmentID)

# Mask Mode 5 means paint only allowed inside one segment
# https://discourse.slicer.org/t/how-can-i-set-masking-settings-on-a-segment-editor-effect-in-python/4406/2
segmentEditorNode.SetMaskMode(5)

# Avoid overwrite of overlapping segments
segmentEditorNode.SetOverwriteMode(slicer.vtkMRMLSegmentEditorNode.OverwriteNone)

# Thresholding

## Read file with thresholding values

In [21]:
grey_df = pd.read_csv(segments_greyvalues_file)

segments_greyvalues = grey_df.to_dict('list')
segments_greyvalues

{'Bone': [8654.02001953125, 65535.0]}

## Create segments by thresholding

In [22]:
ps.segmentation.segments_by_thresholding(segments_greyvalues, 
                                         segmentationNode,
                                         segmentEditorNode,
                                         segmentEditorWidget)

Set segments color

In [23]:
segments_color = {
    "Bone": (0.9450980392156862, 0.8392156862745098, 0.5686274509803921), # "Bone" color in Slicer
    }

segments_color

{'Bone': (0.9450980392156862, 0.8392156862745098, 0.5686274509803921)}

In [24]:
ps.segmentation.set_segments_color(segments_color, segmentationNode)

In [28]:
# Monitor Memory in Slicer
import psutil, os
print(psutil.Process(os.getpid()).memory_info().rss / (1024**3), "GB used")

17.039264678955078 GB used


# Operation on segments

## Manual fix of the segmentation

Sometimes it might be necessary to remove speckles at the image boundaries. If so, use the `scissor` tool in the `Segment Editor` before proceeding with the rest of the script. 

## Remove small islands

REMOVE_SMALL_ISLANDS operation from the [SegmentEditorIslandsEffect](https://github.com/Slicer/Slicer/blob/294ef47edbac2ccb194d5ee982a493696795cdc0/Modules/Loadable/Segmentations/EditorEffects/Python/SegmentEditorIslandsEffect.py#L402)

In [31]:
# segment_name = 'Bone'
# minimum_size = 20 #number of voxels

In [32]:
# ps.segmentation.remove_small_islands(minimum_size, 
#                                      segment_name, 
#                                      segmentEditorNode,
#                                      segmentEditorWidget)

## Smoothing thresholded segment

In [25]:
segment_name = 'Bone'
gaussiaSD_mm = 0.001 #units are in millimiters

In [26]:
ps.segmentation.gaussian_smoothing(gaussiaSD_mm, 
                                   segment_name, 
                                   segmentEditorNode, 
                                   segmentEditorWidget)

# Export Bone segment

# Create a new temporary segmentation node with only the `Bone` segment

In [49]:
seg_name = 'Bone'

# Create a new temporary segmentation node
temp_seg_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")

# Get the segment from the original node
segment = segmentationNode.GetSegmentation().GetSegment(seg_name)

# Add a new empty segment to the temporary node and copy the original segment
temp_seg_node.GetSegmentation().AddSegment(segment)

True

## Export as seg.nrrd file

In [48]:
filename_output = seg_name + '.seg.nrrd'

# Save the single segment to a .seg.nrrd file
slicer.util.saveNode(temp_seg_node, str(output_directory / filename_output))

# Optionally, remove the temporary node from the scene
#slicer.mrmlScene.RemoveNode(temp_seg_node)

## Convert all segments to model nodes

Get closed surface representation of the segment, from [slicer scripting repository](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-nodes-warped-by-transform-sequence)

In [50]:
temp_seg_node.CreateClosedSurfaceRepresentation()

True

In [51]:
shNode = slicer.mrmlScene.GetSubjectHierarchyNode()
exportFolderItemId = shNode.CreateFolderItem(shNode.GetSceneItemID(), "Segments")
slicer.modules.segmentations.logic().ExportAllSegmentsToModels(temp_seg_node, exportFolderItemId)

True

In [53]:
# Optionally, remove the temporary node from the scene
slicer.mrmlScene.RemoveNode(temp_seg_node)

In [54]:
# Monitor Memory in Slicer
import psutil, os
print(psutil.Process(os.getpid()).memory_info().rss / (1024**3), "GB used")

32.05612564086914 GB used


## Decimate model node from segmentation node

Get `VTK Polydata` from model

In [55]:
nodename = 'Bone'
modelNode = slicer.util.getNode(nodename)

Use pyvista to get faces and vertices of the `VTK PolyData` object

In [56]:
import pyvista as pv

In [57]:
model_pv = pv.wrap(modelNode.GetPolyData())
vertices = model_pv.points
faces = model_pv.regular_faces

Import pyvista object to pymeshlab, which performs more efficient decimation than VTK

Check how to [import mesh from arrays in meshlab](https://pymeshlab.readthedocs.io/en/0.1.9/tutorials/import_mesh_from_arrays.html)

In [58]:
import pymeshlab 
# create a new Mesh with the two arrays
model_ml = pymeshlab.Mesh(vertices, faces)

# create a new MeshSet
mesh_set = pymeshlab.MeshSet()

# add the mesh to the MeshSet
mesh_set.add_mesh(model_ml, "mesh")

print('input mesh has', model_ml.vertex_number(), 'vertex and', model_ml.face_number(), 'faces')

input mesh has 769410 vertex and 1544452 faces


Decimate mesh with `pymeshlab`, from [stackoverflow](https://stackoverflow.com/questions/75169329/how-to-use-pymeshlab-to-reduce-number-of-faces-to-a-guaranteed-specific-number)

In [ ]:
decimation_factor = 0.2

#Target number of vertex
TARGET = round(model_ml.vertex_number() * decimation_factor)

#Reduce to TARGET. Sometimes will fall into TARGET-1
mesh_set.meshing_decimation_quadric_edge_collapse(targetfacenum=TARGET, preservenormal=True)

model_ml = mesh_set.current_mesh()
print('output mesh has', model_ml.vertex_number(), 'vertex and', model_ml.face_number(), 'faces')

Load decimated model to Slicer

In [ ]:
vertices = model_ml.vertex_matrix()
faces = model_ml.face_matrix()
model_decimated = pv.PolyData.from_regular_faces(vertices, faces)

decimatedNode = slicer.modules.models.logic().AddModel(model_decimated)

## Export decimated model from segmentation node

Export pymeshlab object to .stl file

In [ ]:
filename_output = 'Bone.stl'

slicer.util.exportNode(decimatedNode, output_directory / filename_output)